In [43]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI, P_PHONE
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np

In [2]:
request_url = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY_WAPI}&q=mexico city&days=1&aqi=no&alerts=no"
response = requests.get(request_url).json()
response

## Creando dataframes

In [9]:
def get_forecast(response, i):
    fecha = response["forecast"]["forecastday"][0]["hour"][i]['time'].split()[0]
    hora = int(response["forecast"]["forecastday"][0]["hour"][i]['time'].split()[1].split(':')[0])
    condicion = response["forecast"]["forecastday"][0]["hour"][i]['condition']['text']
    temperatura = response["forecast"]["forecastday"][0]["hour"][i]['temp_c']
    rain = response["forecast"]["forecastday"][0]["hour"][i]['will_it_rain']
    prob_rain = response["forecast"]["forecastday"][0]["hour"][i]['chance_of_rain']
    
    return fecha, hora, condicion, temperatura, rain, prob_rain


    

In [17]:
datos = []

for i in tqdm(range( len(response["forecast"]["forecastday"][0]["hour"])), colour ="purple"):
    datos.append(get_forecast(response, i))
    

/Users/jibo16/environments/weather_forecast/lib/python3.11/site-packages/tqdm/std.py:161: TqdmWarning: Unknown colour (purple); valid choices: [hex (#00ff00), BLACK, RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE]
  self.colour = colour
100%|████████████████████████████████████████| 24/24 [00:00<00:00, 98016.84it/s]


In [18]:
datos

[('2023-11-27', 0, 'Partly cloudy', 14.8, 0, 0),
 ('2023-11-27', 1, 'Cloudy', 14.5, 0, 0),
 ('2023-11-27', 2, 'Cloudy', 14.3, 0, 0),
 ('2023-11-27', 3, 'Cloudy', 14.1, 0, 0),
 ('2023-11-27', 4, 'Overcast', 12.6, 0, 0),
 ('2023-11-27', 5, 'Overcast', 12.5, 0, 0),
 ('2023-11-27', 6, 'Overcast', 12.5, 0, 0),
 ('2023-11-27', 7, 'Overcast', 12.7, 0, 0),
 ('2023-11-27', 8, 'Overcast', 12.9, 0, 0),
 ('2023-11-27', 9, 'Overcast', 13.1, 0, 0),
 ('2023-11-27', 10, 'Overcast', 13.7, 0, 0),
 ('2023-11-27', 11, 'Overcast', 15.1, 0, 0),
 ('2023-11-27', 12, 'Overcast', 17.0, 0, 0),
 ('2023-11-27', 13, 'Overcast', 18.6, 0, 0),
 ('2023-11-27', 14, 'Cloudy', 19.9, 0, 0),
 ('2023-11-27', 15, 'Partly cloudy', 21.0, 0, 0),
 ('2023-11-27', 16, 'Partly cloudy', 21.5, 0, 0),
 ('2023-11-27', 17, 'Partly cloudy', 21.0, 0, 0),
 ('2023-11-27', 18, 'Partly cloudy', 20.2, 0, 0),
 ('2023-11-27', 19, 'Overcast', 18.6, 0, 0),
 ('2023-11-27', 20, 'Overcast', 16.8, 0, 0),
 ('2023-11-27', 21, 'Overcast', 15.4, 0, 0),
 ('

In [31]:
cols = ['fecha', 'hora', 'condicion', 'temperatura', 'lluvia', 'prob_lluvia']
df = pd.DataFrame(datos,columns = cols)
df

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia
0,2023-11-27,0,Partly cloudy,14.8,0,0
1,2023-11-27,1,Cloudy,14.5,0,0
2,2023-11-27,2,Cloudy,14.3,0,0
3,2023-11-27,3,Cloudy,14.1,0,0
4,2023-11-27,4,Overcast,12.6,0,0
5,2023-11-27,5,Overcast,12.5,0,0
6,2023-11-27,6,Overcast,12.5,0,0
7,2023-11-27,7,Overcast,12.7,0,0
8,2023-11-27,8,Overcast,12.9,0,0
9,2023-11-27,9,Overcast,13.1,0,0


## Template

In [44]:
msg = f'\n \n El pronostico del tiempo hoy a las {df["hora"][15]} es de {df["condicion"][15]}'
msg

'\n \n El pronostico del tiempo hoy a las 15 es de Partly cloudy'

# Mensaje msm por twilio

In [42]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid,auth_token)
message = client.messages \
                .create(
                    body = msg,
                    from_=PHONE_NUMBER,
                    to= P_PHONE
                )
print('Mensaje enviado' + message.sid)

Mensaje enviadoSM06d431f7b61ee2f72835bd531dcbd00a
